In [1]:
#!/usr/bin/env python

# Work with Ros and Jupyter library 
import rospy
import actionlib
import assignment_2_2023.msg
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import tf

# Import the mesasages that we are gonna need
from std_srvs.srv import *
from std_srvs.srv import Empty
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from actionlib_msgs.msg import GoalStatus
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningFeedback, PlanningActionFeedback
from assignment_2_2023.msg import RobotPV

# Import the widgets use in the interface
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, Output, interact

# To display the events library
from IPython.display import display
from tf.transformations import quaternion_matrix
from matplotlib.animation import FuncAnimation

#%matplotlib inline
%matplotlib widget

# VARIABLE AND PARAMETERS OF THE NODE

In [2]:
# Global variables
client = None
publisher = None
subscriber = None

# Variable to make the graph
x_data = []
y_data = []

# Parameter where are stored the reached goals
reached_target = []

# Parameter where are stored the cancel goals
cancel_target = []

# Parameter where are store the target that are sent
target_sent_x = []
target_sent_y = []

In [3]:
def make_box_layout_button():
    return widgets.Layout(margin='100px 5px 5px 3px')

def make_box_layout():
    return widgets.Layout(margin='10px 10px 10px 0px')
 
def make_box_title():
    return widgets.Layout(margin='10px 10px 10px 250px')

# BUTTON CLASS

In [4]:
class Button:
    def __init__(self):
        self.cancel = widgets.Button(value=False, description='Cancel Goal', disabled=False, button_style='danger', icon='times')
        self.cancel.layout = make_box_layout_button()
        self.send_goal = widgets.Button(value=False, description='Send Goal', disabled=False, button_style='success', icon='play')
        self.target = widgets.Button(value=False, description='Targets history', disabled=False, button_style='info')
        self.reached_goal = widgets.Button(value=False, description='Reached target', disabled=False, button_style='info')
        self.cancel_goal = widgets.Button(value=False, description='Cancel Target', disabled=False, button_style='info')
        self.kill = widgets.Button(value=False, description='Reset enviroment', disabled=False, button_style = 'warning')
        self.x_input = widgets.BoundedFloatText(description='x:', value=False, min=-9, max=9,bar_style = 'info')
        self.y_input = widgets.BoundedFloatText(description='y:', value=False, min=-9, max=9, bar_style = 'info')
        
        self.send_goal.on_click(self.goal_definition)
        self.cancel.on_click(self.on_cancel_clicked)
        self.reached_goal.on_click(self.show_reached)
        self.cancel_goal.on_click(self.show_cancel)
        self.kill.on_click(self.reset_env)
        
    def goal_definition(self,b):
        global client, x, y, sendGoal
        client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
        client.wait_for_server()
        x = self.x_input.value
        y = self.y_input.value
        target_sent_x.append(x)
        target_sent_y.append(y)
        sendGoal = True
        
        target = assignment_2_2023.msg.PlanningGoal()
        target.target_pose.pose.position.x = x
        target.target_pose.pose.position.y = y
        rospy.set_param('/des_pos_x', x)
        rospy.set_param('/des_pos_y', y)
        client.send_goal(target)
        print("Goal was sent!")

    def on_cancel_clicked(self,b):
        global cancel_target, client
        if (len(target_sent_x)==0):
            print('No goal has been set')
        else:
            client.cancel_goal()
            x = rospy.get_param('/des_pos_x')
            y = rospy.get_param('/des_pos_y')
            cancel_target.append([x, y])
            print ("Goal was cancel")

    def show_reached(self,b):
        if (len(reached_target)== 0):
            print("No target has been sent")
        else:
            print("Reached target: ")
            for i in range(len(reached_target)):
                print(str(reached_target[i]))

    def show_cancel(self,b):
        if (len(cancel_target)== 0):
            print("No target has been cancel")
        else:
            print("Delete target: ")
            for i in range(len(cancel_target)):
                print(str(cancel_target[i]))
    
    def reset_env(self,b):
        rospy.wait_for_service('/gazebo/reset_world')
        try:
            reset_world = rospy.ServiceProxy('/gazebo/reset_world', Empty)
            reset_world()
        except rospy.ServiceException:
            print("something")

# VISUALISER CLASS

In [5]:
class Visualiser:
    global sendGoal
    def __init__(self):
        
        output = widgets.Output()
        plt.ioff()
        self.fig, (self.ax1, self.ax2) = plt.subplots(1,2,figsize=(7,3),constrained_layout=True)
        self.fig.canvas.toolbar_position = 'bottom'
        plt.ion()
        
        # Plot of the cross in the new target
        self.new_target = False
        sendGoal = False
        
        
        with output:
            display(self.fig.canvas)
        
        # Bar graph and button class
        self.bar_data = [0,0]
        self.bar_labels = ['Reached','Cancel']
        self.bars = self.ax2.bar(self.bar_labels, self.bar_data)
        
        # Title of the subplot
        self.ax1.set_title('Robot Position')
        self.ax2.set_title('Bar Graph')
        
        plt.ioff()
        self.ln, = self.ax1.plot([], [], 'ro')
        self.xdata, self.ydata = [] , []
        plt.ion()
        
        buttons = Button()
         
        # Layout 
        line_one = widgets.HBox([buttons.x_input, buttons.y_input, buttons.send_goal])
        
        col1 = widgets.VBox([buttons.cancel,buttons.reached_goal, buttons.cancel_goal, buttons.kill])
        col1.layout = make_box_layout()
        col2 = widgets.Box([output])
        col = widgets.HBox([col1,col2])
        title = widgets.HTML(value="<h2>Assignment Interface </h2>")
        title.layout = make_box_title()
        
        interface = widgets.VBox([title,line_one, col])
        
        display(interface)
        
    def plot_init(self):
        self.ax1.set_xlim(-10, 10)
        self.ax1.set_ylim(-10, 10)
        self.ax2.set_xlim()
        self.ax2.set_ylim(0,5)
        return self.ln, self.bars
        
    def odom_callback(self, msg):
        self.ydata.append(msg.pose.pose.position.y)
        self.xdata.append(msg.pose.pose.position.x)
        cancel = len(cancel_target)
        reach = len(reached_target)
        new_data = [reach,cancel]
        self.update_bar(new_data)

    def update_plot(self, frame):

        self.ln.set_data(self.xdata, self.ydata)
        self.bars[0].set_height(self.bar_data[0])
        self.bars[1].set_height(self.bar_data[1])

        return self.ln, self.bars
    
    def update_bar(self, new_data):
        self.bar_data = new_data
    
    

# CALLBACK OF THE SUBSCRIBER

In [6]:
def callbackPositionVelocity(msg):
    global publisher
    
    # Put in a variable the position obtain in subscribing to /odom
    position = msg.pose.pose.position
    velocity = msg.twist.twist.linear
    
    # Position of the x-axis and y-axis for the graph 
    x_data.append(position.x)
    y_data.append(position.y)
    
    
    # Position_velocity have the structure of the RobotPV msg
    position_velocity = RobotPV()
    position_velocity.x = position.x
    position_velocity.y = position.y
    position_velocity.linear_x = velocity.x
    position_velocity.angular_z = velocity.z
    publisher.publish(position_velocity)
    
def status_callback(msg):
    global x, y
    position = msg.feedback.stat
    if position == "Target reached!":
        reached_target.append([x,y])
    

In [7]:
out = widgets.Output()
with out:
    vis = Visualiser()
    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit = True)
    


In [8]:
def main():
    global publisher
    
    rospy.init_node('nodeA_jupyter')
    
    # Create a publisher that publish position and velocity on the topic /position_velocity
    publisher = rospy.Publisher('/position_velocity', RobotPV, queue_size = 1)
    
    # Subscribe to the /odom topic
    rospy.Subscriber('/odom', Odometry, callbackPositionVelocity)
    
    # Subcribe to thr /reaching_goal/feedback topic
    rospy.Subscriber("/reaching_goal/feedback",PlanningActionFeedback,status_callback)
    
    display(out)
    
    

In [9]:
if __name__ == '__main__':
    main()

Output()